In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


c:\Users\Juan Pablo\.conda\envs\p37env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Dataset**

In [7]:
mnist_dataset, mnist_info = tfds.load(name="mnist", 
                                      with_info=True, 
                                      as_supervised=True)

In [9]:
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255. 
    return image, label 

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [10]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [14]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

**Modelo**

In [16]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(output_size, activation="softmax")
])

**Elegimos el optimizer y la loss function**

In [17]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

**Entrenamiento**

In [19]:
NUM_EPOCHS = 10

model.fit(train_data, 
          epochs=NUM_EPOCHS,
          validation_data=(validation_inputs, validation_targets),
          verbose=2
)

Epoch 1/10
540/540 - 1s - loss: 0.0517 - accuracy: 0.9840 - val_loss: 0.0469 - val_accuracy: 0.9847 - 1s/epoch - 3ms/step
Epoch 2/10
540/540 - 1s - loss: 0.0436 - accuracy: 0.9864 - val_loss: 0.0377 - val_accuracy: 0.9885 - 1s/epoch - 2ms/step
Epoch 3/10
540/540 - 1s - loss: 0.0345 - accuracy: 0.9893 - val_loss: 0.0427 - val_accuracy: 0.9858 - 1s/epoch - 2ms/step
Epoch 4/10
540/540 - 1s - loss: 0.0299 - accuracy: 0.9907 - val_loss: 0.0443 - val_accuracy: 0.9835 - 1s/epoch - 2ms/step
Epoch 5/10
540/540 - 1s - loss: 0.0265 - accuracy: 0.9916 - val_loss: 0.0281 - val_accuracy: 0.9907 - 1s/epoch - 2ms/step
Epoch 6/10
540/540 - 1s - loss: 0.0220 - accuracy: 0.9929 - val_loss: 0.0231 - val_accuracy: 0.9918 - 1s/epoch - 3ms/step
Epoch 7/10
540/540 - 1s - loss: 0.0187 - accuracy: 0.9939 - val_loss: 0.0182 - val_accuracy: 0.9942 - 1s/epoch - 3ms/step
Epoch 8/10
540/540 - 1s - loss: 0.0175 - accuracy: 0.9943 - val_loss: 0.0218 - val_accuracy: 0.9918 - 1s/epoch - 2ms/step
Epoch 9/10
540/540 - 1s 

**Testeando el modelo**

In [20]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 317ms/step - loss: 0.0885 - accuracy: 0.9784
